<a href="https://colab.research.google.com/github/boskidisanalysis/Movies_Recomendation/blob/main/Semantic_Movie_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 15.6 MB/s 
     |████████████████████████████████| 85 kB 2.4 MB/s 
     |████████████████████████████████| 182 kB 74.0 MB/s 
     |████████████████████████████████| 7.6 MB 50.8 MB/s 
     |████████████████████████████████| 1.3 MB 57.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=7f482c91ea59e8c5c84ea73db08d862d32ac75076a5001b42a024f07b1546566
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [2]:
def get_zip_file(url):
  # importing the requests module
  import requests
  print('Downloading started')
  

  # Downloading the file by sending the request to the URL
  req = requests.get(url)

  # Split URL to get the file name
  filename = url.split('/')[-1]

  # Writing the file to the local file system
  with open(filename,'wb') as output_file:
    output_file.write(req.content)
  print('Downloading Completed')

In [3]:
url = 'https://files.grouplens.org/datasets/movielens/ml-25m.zip'
get_zip_file(url)

In [4]:
!unzip ml-25m.zip

Archive:  ml-25m.zip
   creating: ml-25m/
  inflating: ml-25m/tags.csv         
  inflating: ml-25m/links.csv        
  inflating: ml-25m/README.txt       
  inflating: ml-25m/ratings.csv      
  inflating: ml-25m/genome-tags.csv  
  inflating: ml-25m/genome-scores.csv  
  inflating: ml-25m/movies.csv       


In [5]:
import pandas as pd 
import numpy as np
data = pd.read_csv('/content/ml-25m/movies.csv')

In [6]:
data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
import re
def remove_pars(x):
  x = str(x)
  return re.sub('[()]',"",x)

In [8]:
titles = [remove_pars(i) for i in data['title']]

In [9]:
def remove_pipes(x):
  x=str(x)
  return re.sub('\|', " ",x)

In [10]:
genres = [remove_pipes(i) for i in data['genres']]

In [11]:
def remove_nulls(a,b,i):
  string_m = a[i] + ' '+ b[i]
  return re.sub('\(no genres listed\)',"", string_m)

In [12]:
input_string = [remove_nulls(titles, genres, i) for i in range(len(genres)) ]

In [13]:
input_string[:10]

['Toy Story 1995 Adventure Animation Children Comedy Fantasy',
 'Jumanji 1995 Adventure Children Fantasy',
 'Grumpier Old Men 1995 Comedy Romance',
 'Waiting to Exhale 1995 Comedy Drama Romance',
 'Father of the Bride Part II 1995 Comedy',
 'Heat 1995 Action Crime Thriller',
 'Sabrina 1995 Comedy Romance',
 'Tom and Huck 1995 Adventure Children',
 'Sudden Death 1995 Action',
 'GoldenEye 1995 Action Adventure Thriller']

In [14]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [15]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2',)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [16]:
movies_embeddings = embedder.encode(input_string, convert_to_tensor = True)

In [18]:
top_k = min(10, len(input_string))

In [20]:
query = ' A funny action movie with soldiers' #@param 

In [21]:
from sentence_transformers import util
query_embedding = embedder.encode(query, convert_to_tensor=True)

# We use cosine-similarity and torch.topk to find the highest 5 scores
cos_scores = util.cos_sim(query_embedding, movies_embeddings)[0]
top_results = torch.topk(cos_scores, k=top_k)

print("\n\n======================\n\n")
print("Query:", query)
print("\nTop 10 most similar sentences in corpus:")

for score, idx in zip(top_results[0].cpu(), top_results[1].cpu()):
  print(data['title'].iloc[idx.numpy()]," || ",data['genres'].iloc[idx.numpy()] ,' || ', f"(Score: {score:.4f})")





Query:  A funny action movie with soldiers

Top 10 most similar sentences in corpus:
Paper Soldiers (2002)  ||  Action|Comedy  ||  (Score: 0.7461)
A Soldier's Story (2015)  ||  Action|Drama  ||  (Score: 0.7310)
Dad's Army (2016)  ||  Comedy  ||  (Score: 0.7222)
Dad's Army (1971)  ||  Comedy  ||  (Score: 0.7107)
In the Army Now (1994)  ||  Comedy|War  ||  (Score: 0.7099)
Troops (1997)  ||  Comedy|Sci-Fi  ||  (Score: 0.7088)
The Good Soldier Svejk (1955)  ||  Animation|Comedy  ||  (Score: 0.7006)
Call Out the Marines (1942)  ||  Comedy  ||  (Score: 0.6925)
Tales of the Stinking Military Service (1994)  ||  Comedy  ||  (Score: 0.6888)
Army of One (2016)  ||  Comedy  ||  (Score: 0.6825)


In [41]:
# """
# # Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
hits = util.semantic_search(query_embedding, movies_embeddings, top_k=top_k)
hits = hits[0]      #Get the hits for the first query
for hit in hits:
  print(data['title'].iloc[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
# """

Paper Soldiers (2002) (Score: 0.7461)
A Soldier's Story (2015) (Score: 0.7310)
Dad's Army (2016) (Score: 0.7222)
Dad's Army (1971) (Score: 0.7107)
In the Army Now (1994) (Score: 0.7099)
Troops (1997) (Score: 0.7088)
The Good Soldier Svejk (1955) (Score: 0.7006)
Call Out the Marines (1942) (Score: 0.6925)
Tales of the Stinking Military Service (1994) (Score: 0.6888)
Army of One (2016) (Score: 0.6825)


In [42]:
hits = util.semantic_search(query_embedding, movies_embeddings, top_k=top_k)
hits=hits[0]

In [37]:
 pd.DataFrame(hits[0])

,corpus_id,score
0,25629,0.746083
1,59386,0.730971
2,38101,0.722162
3,37196,0.710685
4,468,0.709896
5,50774,0.708774
6,62015,0.700633
7,34882,0.692465
8,52727,0.688779
9,43388,0.682534


In [40]:
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

Downloading:   0%|          | 0.00/794 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [43]:
##### Re-Ranking #####
# Now, score all retrieved passages with the cross_encoder
cross_inp = [[query, input_string[hit['corpus_id']]] for hit in hits]
cross_scores = cross_encoder.predict(cross_inp)

In [44]:
# Sort results by the cross-encoder scores
for idx in range(len(cross_scores)):
  hits[idx]['cross-score'] = cross_scores[idx]

In [47]:
hits

[{'corpus_id': 25629, 'score': 0.7460829615592957, 'cross-score': 5.3841863},
 {'corpus_id': 59386, 'score': 0.7309714555740356, 'cross-score': 1.7873832},
 {'corpus_id': 38101, 'score': 0.7221620082855225, 'cross-score': -3.779891},
 {'corpus_id': 37196, 'score': 0.710684597492218, 'cross-score': -2.7915728},
 {'corpus_id': 468, 'score': 0.7098960280418396, 'cross-score': 0.65545446},
 {'corpus_id': 50774, 'score': 0.7087735533714294, 'cross-score': 0.5142287},
 {'corpus_id': 62015, 'score': 0.7006334662437439, 'cross-score': 2.1947165},
 {'corpus_id': 34882, 'score': 0.6924653649330139, 'cross-score': -3.628242},
 {'corpus_id': 52727, 'score': 0.6887786388397217, 'cross-score': -0.8405144},
 {'corpus_id': 43388, 'score': 0.6825342178344727, 'cross-score': -1.8916543}]

In [48]:
hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)


In [51]:
hits_df = pd.DataFrame(hits)

In [52]:
hits_df

,corpus_id,score,cross-score
0,25629,0.746083,5.384186
1,62015,0.700633,2.194716
2,59386,0.730971,1.787383
3,468,0.709896,0.655454
4,50774,0.708774,0.514229
5,52727,0.688779,-0.840514
6,43388,0.682534,-1.891654
7,37196,0.710685,-2.791573
8,34882,0.692465,-3.628242
9,38101,0.722162,-3.779891


In [53]:
data.iloc[hits_df['corpus_id']]

,movieId,title,genres
25629,124168,Paper Soldiers (2002),Action|Comedy
62015,207576,The Good Soldier Svejk (1955),Animation|Comedy
59386,200746,A Soldier's Story (2015),Action|Drama
468,473,In the Army Now (1994),Comedy|War
50774,181755,Troops (1997),Comedy|Sci-Fi
52727,185919,Tales of the Stinking Military Service (1994),Comedy
43388,165897,Army of One (2016),Comedy
37196,152105,Dad's Army (1971),Comedy
34882,146457,Call Out the Marines (1942),Comedy
38101,154065,Dad's Army (2016),Comedy


In [55]:
[hit['corpus_id'] for hit in hits]

[25629, 62015, 59386, 468, 50774, 52727, 43388, 37196, 34882, 38101]

In [56]:
def search_with_reranking(query:str):
  #Semantic Search
  query_embedding = embedder.encode(query, convert_to_tensor=True)
  hits = util.semantic_search(query_embedding, movies_embeddings, top_k=top_k)
  hits = hits[0]

  #Re-Ranking with Cross-Encoder
  cross_inp = [[query, input_string[hit['corpus_id']]] for hit in hits]
  cross_scores = cross_encoder.predict(cross_inp)
  # Sort results by the cross-encoder scores
  for idx in range(len(cross_scores)):
    hits[idx]['cross-score'] = cross_scores[idx]
  hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
  hits_df = pd.DataFrame(hits)
  return data[['title','genres']].iloc[hits_df['corpus_id']]


In [57]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.8 MB 4.8 MB/s 
     |████████████████████████████████| 278 kB 68.6 MB/s 
     |████████████████████████████████| 54 kB 4.2 MB/s 
     |████████████████████████████████| 106 kB 74.0 MB/s 
     |████████████████████████████████| 2.3 MB 59.7 MB/s 
     |████████████████████████████████| 55 kB 4.6 MB/s 
     |████████████████████████████████| 213 kB 76.1 MB/s 
     |████████████████████████████████| 56 kB 6.6 MB/s 
     |████████████████████████████████| 84 kB 3.9 MB/s 
     |████████████████████████████████| 84 kB 4.9 MB/s 
     |████████████████████████████████| 64 kB 4.0 MB/s 
     |████████████████████████████████| 80 kB 12.0 MB/s 
     |████████████████████████████████| 68 kB 9.7 MB/s 
     |████████████████████████████████| 68 kB 8.7 MB/s 
     |████████████████████████████████| 68 kB 9.0 MB/s 
     |████████████████████████████████| 68 kB 9.2 

In [58]:
import gradio as gr

# We instantiate the Textbox class
textbox = gr.Textbox(label="Type your search here:", placeholder="Search for a film", lines=2)

gr.Interface(fn=search_with_reranking, inputs=textbox, outputs="dataframe").launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>